# Module 2 Project - Housing Price Regression

## Scope and Business Understanding
I will be building a multivariate regression on the KC Housing Dataset with the goal of predicting house sale price as accurately as possible. This exercise will be completed in the context of a private equity / real estate investment firm looking for diligence and valuation support. Ideally, this regression will help identify statistically significant variables to target during due diligence processes to ensure time is spent researching aspects of a house that are meaningful to price. Additionally, if the data supports it, firms will be able to leverage this data to determine which potential future projects might be correlated with higher sales prices to guide renovation work. Ultimately, using the tool for buyers and sellers, this tool can be used to minimize risk and help improve return realization. 

I will aim to answer the following three primary questions:
1. What does the current KC housing market look like and what do the majority of houses have in common
2. Are there specific times (years, months, days) that might be related with a higher sale price?
3. Are provided rankings (grade, condition, etc.) trustworthy and are they actually relevant to predicting sale price?

## Overview of Process:
General process steps are as followed:
1. Load dataset and handle data issues (missing values, weird values, column data types, etc.)
2. Identify continuous, ordinal categorical, and non-ordinal categorical variables
3. Drop continuous variables that do not meet linearity requirements
4. Handle multi-collinearity amongst predictors
5. Handle categorical variables - treat ordinal categorical variables as single columns and encode non-ordinal categorical columns
6. Use stepwise selection to select features that meet p-value thresholds 
7. Run baseline model with these features 
8. Evaluate regression diagnostics and check assumptions
9. Continue preprocessing / transforming to improve results

## Data Understanding

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.api as sms
import scipy.stats as stats
%matplotlib inline

In [2]:
# set style of graphs and visualizations
plt.style.use('ggplot')

In [3]:
# import raw dataset
raw_df = pd.read_csv('kc_house_data.csv')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [4]:
# drop id column as not relevant to analysis
clean_df = raw_df.drop('id', axis=1)

In [5]:
# check for missing values
clean_df.isna().sum()

date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

We can see we are missing values in three columns - waterfront, yr_renovated, and view, with far more missing out of waterfront and yr_renovated.

In [6]:
# calc percentage of data missing
waterfront_missing = round(clean_df['waterfront'].isna().sum() / len(clean_df), 4)
yr_renovated_missing = round(clean_df['yr_renovated'].isna().sum() / len(clean_df), 4)
view_missing = round(clean_df['view'].isna().sum() / len(clean_df), 4)

# print percentages
print(f'Missing waterfront values comprise {waterfront_missing * 100}% of total data')
print(f'Missing yr_renovated values comprise {yr_renovated_missing * 100}% of total data')
print(f'Missing view values comprise {view_missing * 100}% of total data')

Missing waterfront values comprise 11.0% of total data
Missing yr_renovated values comprise 17.79% of total data
Missing view values comprise 0.29% of total data


In [7]:
# explore other values in these columns
print('waterfront')
display(clean_df['waterfront'].unique())
print('yr_renovated')
display(clean_df['yr_renovated'].unique())
print('view')
display(clean_df['view'].unique())

waterfront


array([nan,  0.,  1.])

yr_renovated


array([   0., 1991.,   nan, 2002., 2010., 1992., 2013., 1994., 1978.,
       2005., 2003., 1984., 1954., 2014., 2011., 1983., 1945., 1990.,
       1988., 1977., 1981., 1995., 2000., 1999., 1998., 1970., 1989.,
       2004., 1986., 2007., 1987., 2006., 1985., 2001., 1980., 1971.,
       1979., 1997., 1950., 1969., 1948., 2009., 2015., 1974., 2008.,
       1968., 2012., 1963., 1951., 1962., 1953., 1993., 1996., 1955.,
       1982., 1956., 1940., 1976., 1946., 1975., 1964., 1973., 1957.,
       1959., 1960., 1967., 1965., 1934., 1972., 1944., 1958.])

view


array([ 0., nan,  3.,  4.,  2.,  1.])

Looking at the value distribution for the three columns with missing values, can see the following:
* waterfront: only two other values, either 0 or 1. Missing values are likely instances where waterfront=0
* yr_renovated: 0 values refer to non-renovated houses.  Missing values are likely the same
* view: 5 other classes of view

Given these observations, the missing values will be handled in the following manner:
* waterfront: NaNs assigned value of 0
* yr_renovated: NaNs assigned value of 0
* view: NaNs assigned value of column median

In [8]:
# handle missing values
clean_df['waterfront'] = clean_df['waterfront'].fillna(value=0.)
clean_df['yr_renovated'] = clean_df['yr_renovated'].fillna(value=0.)
clean_df['view'] = clean_df['view'].fillna(value=clean_df['view'].median())

In [9]:
# check for duplicates
clean_df.duplicated().sum()

0

We have handled missing values and there are no duplicated entries.  Will move on to checking column types.

In [10]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21597 non-null  object 
 1   price          21597 non-null  float64
 2   bedrooms       21597 non-null  int64  
 3   bathrooms      21597 non-null  float64
 4   sqft_living    21597 non-null  int64  
 5   sqft_lot       21597 non-null  int64  
 6   floors         21597 non-null  float64
 7   waterfront     21597 non-null  float64
 8   view           21597 non-null  float64
 9   condition      21597 non-null  int64  
 10  grade          21597 non-null  int64  
 11  sqft_above     21597 non-null  int64  
 12  sqft_basement  21597 non-null  object 
 13  yr_built       21597 non-null  int64  
 14  yr_renovated   21597 non-null  float64
 15  zipcode        21597 non-null  int64  
 16  lat            21597 non-null  float64
 17  long           21597 non-null  float64
 18  sqft_l

All columns are numeric except for date and sqft_basement which are of type object. These will need to be addressed. 

In [11]:
# check sqft_basement
clean_df['sqft_basement'].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
588.0         1
2350.0        1
946.0         1
1960.0        1
2720.0        1
Name: sqft_basement, Length: 304, dtype: int64

Looking at value counts, there are entries with the value ?. Given there are only 454 of these, we will remove them from the dataset. 

In [12]:
# drop ? values from sqft_basement
clean_df = clean_df.loc[clean_df['sqft_basement'] != '?']

# convert back to number format
clean_df['sqft_basement'] = clean_df['sqft_basement'].astype('float')

In [13]:
clean_df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,10/13/2014,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,0.0,98028,47.7379,-122.233,2720,8062
3,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,3,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,3,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,3,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,1/16/2015,400000.0,3,2.50,1600,2388,2.0,0.0,0.0,3,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287
